<div style="color:white;
           display:fill;
           border-radius:15px;
           background-color:#189ad3;
           font-size:200%;
           font-family:Arial;
           letter-spacing:0.5px">

<p style="padding: 15px;
          color:white;
          text-align:center;">Do I Need An Umbrella tomorrow?
</p>
</div> 

In [333]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# visual
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# missing data
import missingno as msno



# to split dataset to train and test
from sklearn.model_selection import train_test_split

# Scaling
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Data imputation
!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE 

import seaborn as sns
import matplotlib.pyplot as plt
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
from sklearn.metrics import confusion_matrix

#Import Library for Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [334]:
# Loading Data
df = pd.read_csv("/content/drive/MyDrive/weatherAUS.csv")

In [335]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [336]:
df.head(4)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No


In [337]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

<div style="color:white;
           display:fill;
           border-radius:15px;
           background-color:#189ad3;
           font-size:250%;
           font-family:Arial;
           letter-spacing:0.5px">

<p style="padding: 15px;
          color:white;
          text-align:center;">Missing values in data set
</p>
</div>

# Missing values

#### <span style="font-family: Arial;font-size:1.2em;color:#0e92ea">

In [338]:
# missing values
missing_value = 100 * df.isnull().sum()/len(df)
missing_value = missing_value.reset_index()
missing_value.columns = ['variables','missing values in percentage']
missing_value = missing_value.sort_values('missing values in percentage',ascending=False)

# barplot
fig = px.bar(missing_value, y='missing values in percentage',x='variables',title='Missing values % in each column',
             template='none',text='missing values in percentage');
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')



fig.show()

In [339]:
# stats
round(df.describe(),2)

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,143975.00,144199.00,142199.00,82670.00,75625.00,135197.00,143693.00,142398.00,142806.00,140953.00,130395.00,130432.00,89572.00,86102.00,143693.00,141851.00
mean,12.19,23.22,2.36,5.47,7.61,40.04,14.04,18.66,68.88,51.54,1017.65,1015.26,4.45,4.51,16.99,21.68
std,6.40,7.12,8.48,4.19,3.79,13.61,8.92,8.81,19.03,20.80,7.11,7.04,2.89,2.72,6.49,6.94
min,-8.50,-4.80,0.00,0.00,0.00,6.00,0.00,0.00,0.00,0.00,980.50,977.10,0.00,0.00,-7.20,-5.40
25%,7.60,17.90,0.00,2.60,4.80,31.00,7.00,13.00,57.00,37.00,1012.90,1010.40,1.00,2.00,12.30,16.60
50%,12.00,22.60,0.00,4.80,8.40,39.00,13.00,19.00,70.00,52.00,1017.60,1015.20,5.00,5.00,16.70,21.10
75%,16.90,28.20,0.80,7.40,10.60,48.00,19.00,24.00,83.00,66.00,1022.40,1020.00,7.00,7.00,21.60,26.40
max,33.90,48.10,371.00,145.00,14.50,135.00,130.00,87.00,100.00,100.00,1041.00,1039.60,9.00,9.00,40.20,46.70


# Data cleaning
## <span style="font-family: Arial;font-size:1.2em;color:#0e92ea">   


In [340]:
# checking for duplicated

sim = df.duplicated() 
sim.sum()

0

In [341]:
# dropping null values of target variable which is just 2.2%
df = df[df['RainTomorrow'].notnull()]

In [342]:
# drop null values of Rain Today variable which is just 2.2%
df = df[df['RainToday'].notnull()]

In [343]:
# knn imputer
knn_imputer = KNNImputer(n_neighbors=3)

In [344]:
# impute missing values for categorical variables

cat_impute =['MinTemp', 'Rainfall',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm','Temp9am',
       'Temp3pm','WindGustDir', 'WindDir9am', 'WindDir3pm','Location','RainToday']

for i in cat_impute:
    df[i] = df[i].fillna(df[i].mode()[0])

In [345]:
df['Date'] = pd.to_datetime(df['Date'])

In [346]:
df.head(4)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No


In [347]:
# null values check
# since I decided to drop the Sunshine I am not treating it
df.isnull().sum()

Date                 0
Location             0
MinTemp              0
MaxTemp            307
Rainfall             0
Evaporation      59694
Sunshine         66805
WindGustDir          0
WindGustSpeed        0
WindDir9am           0
WindDir3pm           0
WindSpeed9am         0
WindSpeed3pm         0
Humidity9am          0
Humidity3pm          0
Pressure9am          0
Pressure3pm          0
Cloud9am         52625
Cloud3pm         56094
Temp9am              0
Temp3pm              0
RainToday            0
RainTomorrow         0
dtype: int64

In [348]:
# converting object into the Categorical data type

for feature in df.columns:
    if df[feature].dtype =='object':
        df[feature] = pd.Categorical(df[feature]).codes

In [349]:
# dropped 'MaxTemp' because it highly correlated and 'Sunshine' because it has more than 48% values are missing

X = df[['MinTemp', 'Rainfall',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm','Temp9am',
       'Temp3pm','WindGustDir', 'WindDir9am', 'WindDir3pm','Location','RainToday']]

y = df.pop('RainTomorrow')


In [350]:
# Data split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,random_state=0,stratify=y)

In [351]:
X_train.shape, X_test.shape

((98550, 16), (42237, 16))

In [352]:
y_train.value_counts(1)

0    0.778376
1    0.221624
Name: RainTomorrow, dtype: float64

In [353]:
# Scaling data
sc = MinMaxScaler()

x_train = sc.fit_transform(X_train)
x_test = sc.transform(X_test)

<div style="color:white;
           display:fill;
           border-radius:15px;
           background-color:#189ad3;
           font-size:250%;
           font-family:Arial;
           letter-spacing:0.5px">

<p style="padding: 15px;
          color:white;
          text-align:center;">Models
</p>
</div>

## <span style="font-family: Arial;font-size:1.2em;color:#0e92ea">i. Naive Bayes 

In [354]:
NaiveBayse_model = GaussianNB()

NaiveBayse_model.fit(x_train, y_train)

y_predict_NaiveBayse = NaiveBayse_model.predict(x_test)

NaiveBayse_model_score = round(NaiveBayse_model.score(x_test, y_test)*100,2)
NaiveBayse_model_score_train = round(NaiveBayse_model.score(x_train, y_train)*100 , 2)

print('Accuracy on Test set',NaiveBayse_model_score)
print('Accuracy on Train set',NaiveBayse_model_score_train)

print(confusion_matrix(y_test, y_predict_NaiveBayse))



Accuracy on Test set 81.13
Accuracy on Train set 81.12
[[29470  3407]
 [ 4564  4796]]


In [355]:
LR_model = LogisticRegression(max_iter=1000)
LR_model.fit(x_train, y_train)

y_predict_LR = LR_model.predict(x_test)

LR_model_score = round(LR_model.score(x_test, y_test)*100 , 2)
LR_model_score_train = round(LR_model.score(x_train, y_train)*100 ,2)

print('Accuracy on Test set',LR_model_score)
print('Accuracy on Train set',LR_model_score_train)
print(confusion_matrix(y_test, y_predict_LR))

Accuracy on Test set 84.12
Accuracy on Train set 84.29
[[31160  1717]
 [ 4992  4368]]


# models comparison

In [356]:
Accuracy_score = pd.DataFrame({'Models':['Logistic Regression','Naive Bayse'],'Accuracy on Test set':[LR_model_score,NaiveBayse_model_score],'Accuracy on Training set':[LR_model_score_train,NaiveBayse_model_score_train]})

Accuracy_score = Accuracy_score.sort_values('Accuracy on Test set',ascending=False)

In [357]:
round(Accuracy_score ,2)

,Models,Accuracy on Test set,Accuracy on Training set
0,Logistic Regression,84.12,84.29
1,Naive Bayse,81.13,81.12
